In [ ]:
# !pip install soynlp
# !pip install py-hanspell

In [1]:
import pandas as pd

import re

from soynlp.normalizer import * # 정규화
from hanspell import spell_checker # 네이버 맞춤법 검사기를 이용한 파이썬용 한글 맞춤법 검사 라이브러리 # 네이버 맞춤법 검사기는 최대 500자까지 검사 가능

In [2]:
df = pd.read_csv('../data/webtoon_data.csv')

# 불용어, Nomalize 처리

- 기초적인 전처리
- 크롤링한 데이터일 경우, html, tag 제거
- 숫자, 영어 등 필요하지 않는 문자 및 이모티콘 제거
- "@%*=()/+"과 같은 punctation 문장 부호 제거

In [2]:
punct = "!?~':;[]●○•▫️()-_–—"+'"@−∇'+'∞θα•à−β∅³π‘₹´°£€\×™√²'+'‘´°’’`“”“♭∂ω℉℃'

def Clean_text(text):
    for p in punct: # 위에 있는 punct에 있는 문자가 있을 시, 제거
        text = text.replace(p, '')
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s]) # specials에 있는 단어가 있는 경우, 이에 매칭되는 문자로 대체
    return text.strip() # 좌우 공백 제거

In [3]:
def Korean(text):
    return re.sub('[^0-9ㄱ-ㅎㅏ-ㅣ가-힣\n ]', '', text) # 한글과 숫자 제외하고 지우기
def Korean2(text):
    return re.sub("[^0-9ㄴㅋㅜ가-힣\n ]", "", text) # 최종으로 남길 문자들

In [4]:
def Norm_text(text): # 정규화시키기. 반복되는 단어는 2개로 제한
    tt = ''
    for t in text.split('\n')[:]: # 댓글을 '\n'으로 구분
        i = repeat_normalize(t, num_repeats=2) + '\n'
        tt += i
    tt=tt.rstrip('\n') # 맨뒤의 \n 제거
    return tt

def Rep_text(text): # 단어 통일 시키기
    pattern = '(ㅜ|ㅠ){1,}' 
    text = re.sub(pattern=pattern, repl='ㅜ', string=text) # ㅜorㅠ가 1번이상 사용된 경우 ㅜ로 바꾸기
    pattern = '(ㅌ|ㅎ){1,}' 
    text = re.sub(pattern=pattern, repl='ㅋ', string=text) # ㅌorㅎ가 1번이상 사용된 경우 ㅋ로 바꾸기
    pattern = '(\n){1,}' 
    text = re.sub(pattern=pattern, repl='\n', string=text) # \n가 1번이상 사용된 경우 \n로 바꾸기
    return text

def Rep_word(text): # 한 user가 쓴 댓글 안에서 중복되는 단어 제거
    text = str(text)
    tt = '' # 최종 \n으로 구분되는 데이터 부분
    for sen in text.split('\n'):
        words = sen.split() # 공백 기준으로 단어 분리
        
        unique_words = []
        for word in words:
            if word not in unique_words: # 중복된 단어 제거
                unique_words.append(word)
        processed_text = ' '.join(unique_words)
        processed_text = processed_text + '\n'
        tt += processed_text
    tt=tt.rstrip('\n') # 맨뒤의 \n 제거
    return tt

In [6]:
# 위에 정의한 함수들을 best 댓글과 전체 댓글에 적용
df['preprocessed_best_comment'] = df['best_comment'].apply(lambda x: Clean_text(x))
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Rep_text(x))
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Korean(x))
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Norm_text(x))
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Korean2(x))
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Rep_word(x))

df['preprocessed_comment'] = df['comment'].apply(lambda x: Clean_text(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Rep_text(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Korean(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Norm_text(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Korean2(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Rep_word(x))

In [7]:
# 댓글이 없는 경우를 위해 Null값 치환
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].fillna("\n")
df['preprocessed_comment'] = df['preprocessed_comment'].fillna("\n")

## py - hanspell

py-hanspell은 네이버 맞춤법 검사기를 이용한 파이썬용 한글 맞춤법 검사 라이브러리  

네이버 맞춤법 검사기는 최대 500자까지 검사 가능

+ 김종윤 학우님과 함께 라이브러리 실행할 때 발생한 에러 해결후 사용하였습니다.

In [10]:
def Check_text(text):
    checked_text = ''
    for t in text.split('\n'): # 댓글을 '\n'으로 구분
        try:
            result = spell_checker.check(t)
            checking = result.checked + '\n'
            checked_text += checking
        except:
            checking = t + '\n'
            checked_text += checking # 기존문장 그대로 반환
    checked_text=checked_text[:-1] # 맨뒤의 \n 제거
    return checked_text

In [11]:
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Check_text(x))
df['preprocessed_comment'] = df['comment'].apply(lambda x: Check_text(x))

In [14]:
# df.to_csv('../data/pre_hanspell_data.csv', index=False)
# df = pd.read_csv('../data/pre_hanspell_data.csv')

In [104]:
# 반복된 단어 한번 더 제거
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Rep_word(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Rep_word(x))
# 정규화 한번 더
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Norm_text(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Norm_text(x))
# 
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Clean_text(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Clean_text(x))
# 한국어 숫자 남기기
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Korean2(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Korean2(x))
# 단어 통일
df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Rep_text(x))
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Rep_text(x))

In [10]:
# NaN값이 있는 row 제거
df = df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [12]:
# df.to_csv('../data/pre_data.csv', index=False)

# 사용하지는 않은 함수

In [29]:
# https://apps.timwhitlock.info/emoji/tables/unicode#emoji-modal
# https://ko.wikipedia.org/wiki/%EC%9C%A0%EB%8B%88%EC%BD%94%EB%93%9C_1F000~1FFFF
# https://ko.wikipedia.org/wiki/%ED%95%9C%EC%A4%91%EC%9D%BC_%EA%B4%84%ED%98%B8_%EB%AC%B8%EC%9E%90
# http://www.hipenpal.com/tool/characters-to-unicode-charts-in-korean.php?unicode=92
# https://tonks.tistory.com/28
# 이모티콘 제거하는 함수
target_unicode = ["\u1F600-\u1F64F", # 스마일리
                  "\u1F300-\u1F5FF", # 기호
                  "\u1F680-\u1F6FF", # 트랜스포트 및 심볼
                  "\u1F1E0-\u1F1FF", # 국기
                  "\u2702-\u27B0", # 다른 특수 기호          
                  "\u1F90C-\u1F946","\u1F947-\u1F978","\u1F97A-\u1F9CB","\u1F9CD-\u1F9FF", # 기타 얼굴     
                  "\u1F780-\u1F7D4","\u1F7E0-\u1F7EB","\u1F700-\u1F773",  # 기타       
                  "\u1F3B5","\u2669-\u266C",  # 음표        
                  "\u1F300-\u1F6D2", # 얼굴 하트 등
                  
                  # 여러가지 기호들. 별,하트 제외
                  "\u2600-\u2604", "\u2606-\u2664", "\u2666-\u26FF",
                  "\u2200-\u22FF", # 수학 기호
                  "\u2300-\u23FA", # 여러 가지 기술 기호
                  "\u2100-\u214F", # 글자 변형한 기호
                  "\u2580-\u259F", # 네모 기호
                  
                  # 전각/반각 모양
                  "\uFF01-\uFF9F","\uFFA1-\uFFBF","\uFFC2-\uFFC7","\uFFCA-\uFFCF","\uFFD2-\uFFD7","\uFFDA-\uFFDC","\uFFE0-\uFFE6","\uFFE8-\uFFEE",
                  
                  "\u2460-\u24FF","\u3251-\u325F","\u32B1-\u32BF","\u1F10B","\u2776-\u2789", # circled number   
                  "\u24B6-\u24E9","\u1F150-\u1F15F","\u1F160-\u1F169", # circled letter

                  "\u3001-\u303F", # 한중일 기호 및 구두점
                  "\u3200-\u32FF", # 한중일 괄호 문자
                  "\u31C0-\u31E3","\u2E80-\u2E99", # 한자 획
                  "\u2E9B-\u2EF3", # 부수 보충

                  # 호환용 한글 자모
                  # ㄴ,ㅋ,ㅎ,ㅜ,ㅠ 빼고 자음 모음 날리기
                  "\u3131-\u3133","\u3135-\u314A","\u314C-\u314D","\u314F-\u315B","\u315D-\u315F","\u3161-\u3163","\u3165-\u318E","\uD7B0-\uD7C6",

                  # 일본어 유니코드
                  "\u3040-\u309F", # 히라가나
                  "\u30A0-\u30FF","\u31F0-\u31FF", # 가타카나
                  "\u3105-\u312D","\u31A0-\u31B7", # 주음 부호
                  
                  "\u4E00-\u9FD5", # 한자
                  u"U\0000A720-U\0000A7CA",u"U\0000A7D0-U\0000A7D1",u"U\0000A7D3",u"U\0000A7D5-U\0000A7D9",u"U\0000A7F2-U\0000A7FF", # 라틴 확장
                  u"U\0000FB50-U\0000FBC1",u"U\0000FBD3-U\0000FBFF", # 아랍어
                  u"U\00010330-U\0001034A", # 옛 고딕체 알파벳
                  u"U\00010300-U\00010323",u"U\0001032D-U\0000102F"] # 옛 이탈리아 문자

def Emoji(text):
    emoji_pattern = "[" + "".join(target_unicode) + "]"
    emoji_text = re.sub(emoji_pattern, "", text) #문자 제거
    return emoji_text